In [ ]:
import os
import pandas as pd
os.chdir('..')
from src.regressions import *
from src.helpers import *
from src.statistical_analysis import *
from src.evaluation_metrics import *
os.chdir('notebooks')
from sklearn.model_selection import train_test_split

Just some random test to see whether it works what im writing

In [ ]:
PATH = os.path.join("..","data","Hongkong.csv")
dataset = pd.read_csv(PATH)

In [ ]:
x,y = X_y_from_dataset(dataset)
xtest,xforecast,ytest,yforecast = split(x,y)
len(yforecast)

In [ ]:
beta=least_squares(xtest, ytest)

var=1

confidenceinterval(xtest, beta, var, tolerance)

In [ ]:
a=correlation_test(xtest)
a
